In [1]:
from data_loader import load_dataset, load_challenge, save_challenge
from BirdDataset import BirdDataset
from Bob import Bob
import torch
import numpy as np
from torch.utils.data import DataLoader

Bob(
  (image_convs): Sequential(
    (0): ConvGroup(
      (process): Conv2d(20, 32, kernel_size=(3, 3), stride=(1, 1), padding=same)
      (scale): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (norm): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (droput): Dropout2d(p=0.3, inplace=False)
      (relu): ReLU()
    )
    (1): ConvGroup(
      (process): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=same)
      (scale): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (norm): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (droput): Dropout2d(p=0.3, inplace=False)
      (relu): ReLU()
    )
    (2): ConvGroup(
      (process): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=same)
      (scale): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (norm): BatchNorm2d(32, eps=1e-05, momentum=0.1, 

In [2]:
dataset = load_dataset()

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
left_offset = 7
right_offset = 7
name = 'testbert'
bob_dummy = Bob(left_offset, right_offset)
model_parameters = filter(lambda p: p.requires_grad, bob_dummy.parameters())
params = sum([np.prod(p.size()) for p in model_parameters])
print(bob_dummy, params)


Bob(
  (image_convs): Sequential(
    (0): ConvGroup(
      (process): Conv2d(20, 32, kernel_size=(3, 3), stride=(1, 1), padding=same)
      (scale): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (norm): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (droput): Dropout2d(p=0.3, inplace=False)
      (relu): ReLU()
    )
    (1): ConvGroup(
      (process): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=same)
      (scale): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (norm): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (droput): Dropout2d(p=0.3, inplace=False)
      (relu): ReLU()
    )
    (2): ConvGroup(
      (process): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=same)
      (scale): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (norm): BatchNorm2d(32, eps=1e-05, momentum=0.1, 

In [62]:
train_set = BirdDataset(dataset, True, device, left_offset, right_offset)
scaler = train_set.scaler
test_set = BirdDataset(dataset, False, device, left_offset, right_offset, scaler)
train_loader = DataLoader(train_set, batch_size=128, shuffle=True)
test_loader = DataLoader(test_set, batch_size=128, shuffle=False)

print(len(train_set), len(test_set))

99236 20146


In [68]:
from TrainingRoutine import eval, train, test
import importlib
import TrainingRoutine

importlib.reload(TrainingRoutine)
from TrainingRoutine import eval, train, test, smooth_sample_zero

In [64]:
eval(model=Bob(left_offset, right_offset), train_dataloader=train_loader, test_dataloader=test_loader, device=device, max_epochs=500, name=name)

Epoch 1
-------------------------------


Avg loss: 0.507676, Money saved: 4217.94, Smoothed: 4298.24$

Avg loss: 0.440973, Money saved: 4648.69, Smoothed: 4816.59$

Avg loss: 0.355498, Money saved: 4868.57, Smoothed: 5001.07$

Avg loss: 0.364814, Money saved: 4785.01, Smoothed: 4967.71$

Avg loss: 0.342573, Money saved: 4984.50, Smoothed: 5139.80$

Avg loss: 0.354324, Money saved: 4957.63, Smoothed: 5196.83$

Avg loss: 0.340375, Money saved: 4929.98, Smoothed: 5069.48$

Avg loss: 0.333944, Money saved: 4916.65, Smoothed: 5126.75$

Avg loss: 0.341720, Money saved: 4968.07, Smoothed: 5111.27$

Avg loss: 0.320000, Money saved: 4973.57, Smoothed: 5061.87$

Epoch 11
-------------------------------
Avg loss: 0.337629, Money saved: 4964.53, Smoothed: 5180.23$

Avg loss: 0.317031, Money saved: 5081.61, Smoothed: 5252.91$

Avg loss: 0.331380, Money saved: 5036.86, Smoothed: 5151.46$

Avg loss: 0.319161, Money saved: 5039.21, Smoothed: 5194.01$

Avg loss: 0.341246, Money saved: 4898.07, Smoothed: 5053.47$

Avg loss: 0.325832, Money sav

In [66]:
challenge = load_challenge()
challenge_data = {
    'all': {
        'test_features': [sub.squeeze(0) for sub in np.split(challenge, 16)],
        'test_labels': np.split(np.zeros((16 * 3000), dtype=int), 16),
    }
}
challenge_set = BirdDataset(challenge_data, False, device, left_offset, right_offset, scaler)
challenge_loader = DataLoader(challenge_set, batch_size=30, shuffle=False)

In [67]:
model = Bob(left_offset, right_offset)
model.load_state_dict(torch.load(f'Models/{name}'))
model.eval()

model = model.to(device)

In [69]:
result = np.empty((16 * 3000), dtype=int)
with torch.no_grad():
    for batch, (X, y, i) in enumerate(challenge_loader):
        pred = model(X)
        pred = torch.argmax(pred, axis=1).cpu().numpy()
        pred_smooth = smooth_sample_zero(pred)

        result[batch*challenge_loader.batch_size:batch*challenge_loader.batch_size+len(pred)] = pred_smooth
result = np.reshape(result, (16, 3000))
print(result, result.shape)
save_challenge("smoothrob", result)


dummy_input = train_set[np.random.randint(0, len(train_set))][0].unsqueeze(0)
images, one_d = bob_dummy.get_images(dummy_input)
print(images.shape, one_d.shape)

[0 0 0 0 0 5 0 0 0 0 0 0 0 0 0 0 0 0 0 0 5 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 5 0 0 0 0 0 0 0 0 0 0 0 0 0 0 5 0 0 0 0 0 0 0 0 0]
-----------------------------------
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
-----------------------------------
[0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 0 0 0 1 1 1 1 1 0 0 1 1 1 1]
[0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 0 0 0 1 1 1 1 1 0 0 1 1 1 1]
-----------------------------------
[1 0 0 0 1 1 1 1 1 0 0 1 1 1 1 1 0 0 0 5 5 5 5 5 5 5 5 5 5 5]
[1 0 0 0 1 1 1 1 1 0 0 1 1 1 1 1 0 0 0 1 1 1 1 1 1 1 1 1 1 1]
-----------------------------------
[5 5 0 5 5 0 5 5 5 5 5 5 5 5 5 5 5 5 0 5 0 5 5 0 0 0 0 0 0 0]
[5 5 0 5 5 0 5 5 5 5 5 5 5 5 5 5 5 5 0 5 0 5 5 0 0 0 0 0 0 0]
-----------------------------------
[0 0 0 6 6 6 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 5 5 5 0]
[0 0 0 5 5 5 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 5 5 5 0]
-----------------------------------
[0 0 0 5 5 5 5 5 0 0 0 0 0 0 0 0 0 0 0 0